In [1]:
from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor

### Initializing the MLflow Client

In step 1 of the tutorial, we started an MLflow Tracking Server with:

*host* = **127.0.0.1** 

*port* = **8080**

Let's connect to that uri with the MlflowClient

In [2]:
client = MlflowClient(tracking_uri="http://127.0.0.1:8080")

#### Search Experiments with the MLflow Client API

Let's take a look at the Default Experiment that is created for us.

This safe 'fallback' experiment will store Runs that we create if we don't specify a 
new experiment. 

In [3]:
# Search experiments without providing query terms behaves effectively as a 'list' action

all_experiments = client.search_experiments()

print(all_experiments)

[]


In [5]:
# Extract the experiment name and lifecycle_stage

default_experiment = [
    {"name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage}
    for experiment in all_experiments
    if experiment.name == "Default"
][0]

pprint(default_experiment)

{'lifecycle_stage': 'active', 'name': 'Default'}


In [7]:
# Extract the experiment name and lifecycle_stage

default_experiment = [
    {"name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage}
    for experiment in all_experiments
    if experiment.name == "Apple_Models"
]#[0]

pprint(default_experiment)

[]


### Creating a new Experiment

In this section, we'll:

* create a new MLflow Experiment
* apply metadata in the form of Experiment Tags

In [10]:
experiment_description = (
    "This is the grocery forecasting project. "
    "This experiment contains the produce models for apples."
)

experiment_tags = {
    "project_name": "grocery-forecasting",
    "store_dept": "produce",
    "team": "stores-ml",
    "project_quarter": "Q3-2023",
    "mlflow.note.content": experiment_description,
}

produce_apples_experiment = client.create_experiment(name="Apple_Models", tags=experiment_tags)

RestException: RESOURCE_ALREADY_EXISTS: Experiment 'Apple_Models' already exists.

In [4]:
experiment_description = (
    "This is the test for composed tensor data."
)

experiment_tags = {
    "project_name": "data test",
    "mlflow.note.content": experiment_description,
}

test_experiment = client.create_experiment(name="Test", tags=experiment_tags)

In [5]:
# Use search_experiments() to search on the project_name tag key

test_experiment = client.search_experiments(
    filter_string="tags.`project_name` = 'data test'"
)
print(len(test_experiment))
print(type(test_experiment))
print(type(test_experiment[0]))

pprint(test_experiment)
pprint(test_experiment[0])

1
<class 'mlflow.store.entities.paged_list.PagedList'>
<class 'mlflow.entities.experiment.Experiment'>
[<Experiment: artifact_location='mlflow-artifacts:/275732136076985190', creation_time=1702367152826, experiment_id='275732136076985190', last_update_time=1702367152826, lifecycle_stage='active', name='Test', tags={'mlflow.note.content': 'This is the test for composed tensor data.',
 'project_name': 'data test'}>]
<Experiment: artifact_location='mlflow-artifacts:/275732136076985190', creation_time=1702367152826, experiment_id='275732136076985190', last_update_time=1702367152826, lifecycle_stage='active', name='Test', tags={'mlflow.note.content': 'This is the test for composed tensor data.',
 'project_name': 'data test'}>


In [10]:
# Access individual tag data

print(test_experiment[0].tags["mlflow.note.content"])

This is the test for composed tensor data.


### Running our first model training

In this section, we'll:

* create a synthetic data set that is relevant to a simple demand forecasting task
* start an MLflow run
* log metrics, parameters, and tags to the run
* save the model to the run
* register the model during model logging

#### Synthetic data generator for demand of apples

Keep in mind that this is purely for demonstration purposes. 

The demand value is purely artificial and is deliberately covariant with the features. This is not a particularly realistic real-world scenario (if it were, we wouldn't need Data Scientists!). 

In [26]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


def generate_apple_sales_data_with_promo_adjustment(base_demand: int = 1000, n_rows: int = 5000):
    """
    Generates a synthetic dataset for predicting apple sales demand with seasonality and inflation.

    This function creates a pandas DataFrame with features relevant to apple sales.
    The features include date, average_temperature, rainfall, weekend flag, holiday flag,
    promotional flag, price_per_kg, and the previous day's demand. The target variable,
    'demand', is generated based on a combination of these features with some added noise.

    Args:
        base_demand (int, optional): Base demand for apples. Defaults to 1000.
        n_rows (int, optional): Number of rows (days) of data to generate. Defaults to 5000.

    Returns:
        pd.DataFrame: DataFrame with features and target variable for apple sales prediction.

    Example:
        >>> df = generate_apple_sales_data_with_seasonality(base_demand=1200, n_rows=6000)
        >>> df.head()
    """

    # Set seed for reproducibility
    np.random.seed(9999)

    # Create date range
    dates = [datetime.now() - timedelta(days=i) for i in range(n_rows)]
    dates.reverse()

    # Generate features
    df = pd.DataFrame(
        {
            "date": dates,
            "average_temperature": np.random.uniform(10, 35, n_rows),
            "rainfall": np.random.exponential(5, n_rows),
            "weekend": [(date.weekday() >= 5) * 1 for date in dates],
            "holiday": np.random.choice([0, 1], n_rows, p=[0.97, 0.03]),
            "price_per_kg": np.random.uniform(0.5, 3, n_rows),
            "month": [date.month for date in dates],
        }
    )

    # Introduce inflation over time (years)
    df["inflation_multiplier"] = 1 + (df["date"].dt.year - df["date"].dt.year.min()) * 0.03

    # Incorporate seasonality due to apple harvests
    df["harvest_effect"] = np.sin(2 * np.pi * (df["month"] - 3) / 12) + np.sin(
        2 * np.pi * (df["month"] - 9) / 12
    )

    # Modify the price_per_kg based on harvest effect
    df["price_per_kg"] = df["price_per_kg"] - df["harvest_effect"] * 0.5

    # Adjust promo periods to coincide with periods lagging peak harvest by 1 month
    peak_months = [4, 10]  # months following the peak availability
    df["promo"] = np.where(
        df["month"].isin(peak_months),
        1,
        np.random.choice([0, 1], n_rows, p=[0.85, 0.15]),
    )

    # Generate target variable based on features
    base_price_effect = -df["price_per_kg"] * 50
    seasonality_effect = df["harvest_effect"] * 50
    promo_effect = df["promo"] * 200

    df["demand"] = (
        base_demand
        + base_price_effect
        + seasonality_effect
        + promo_effect
        + df["weekend"] * 300
        + np.random.normal(0, 50, n_rows)
    ) * df[
        "inflation_multiplier"
    ]  # adding random noise

    # Add previous day's demand
    df["previous_days_demand"] = df["demand"].shift(1)
    df["previous_days_demand"].fillna(method="bfill", inplace=True)  # fill the first row

    # Drop temporary columns
    df.drop(columns=["inflation_multiplier", "harvest_effect", "month"], inplace=True)

    return df

In [27]:
# Generate the dataset!

data = generate_apple_sales_data_with_promo_adjustment(base_demand=1_000, n_rows=1_000)

data[-20:]

,date,average_temperature,rainfall,weekend,holiday,price_per_kg,promo,demand,previous_days_demand
980,2023-10-28 09:53:58.549478,34.130183,1.454065,1,0,1.449177,1,1501.802447,1213.085782
981,2023-10-29 09:53:58.549476,32.353643,9.462859,1,0,2.856503,1,1348.951553,1501.802447
982,2023-10-30 09:53:58.549473,18.816833,0.391470,0,0,1.326429,1,1175.352029,1348.951553
983,2023-10-31 09:53:58.549471,34.533012,2.120477,0,0,0.970131,1,1251.385504,1175.352029
984,2023-11-01 09:53:58.549469,23.057202,2.365705,0,0,1.049931,0,991.427049,1251.385504
985,2023-11-02 09:53:58.549467,34.810165,3.089005,0,0,2.035149,0,974.971149,991.427049
986,2023-11-03 09:53:58.549465,29.208905,3.673292,0,0,2.518098,0,1056.249547,974.971149
987,2023-11-04 09:53:58.549462,16.428676,4.077782,1,0,1.268979,0,1381.118915,1056.249547
988,2023-11-05 09:53:58.549460,32.067512,2.734454,1,0,0.762317,0,1358.492007,1381.118915
989,2023-11-06 09:53:58.549458,31.938203,13.883486,0,0,1.153301,0,967.040470,1358.492007


### Train and log the model

We're now ready to import our model class and train a ``RandomForestRegressor``

In [1]:
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Use the fluent API to set the tracking uri and the active experiment
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Sets the current active experiment to the "Apple_Models" experiment and returns the Experiment metadata
apple_experiment = mlflow.set_experiment("Test")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "composed_tensor_test"

# Define an artifact path that the model will be saved to.
artifact_path = "model"

In [11]:
apple_experiment.artifact_location

'mlflow-artifacts:/295829569244001181'

In [12]:
with open(apple_experiment.artifact_location+'/test.txt','w') as file:
    file.write('test success.') 

FileNotFoundError: [Errno 2] No such file or directory: 'mlflow-artifacts:/295829569244001181/test.txt'

In [9]:
import torch
sd = torch.load('../pytorch_model.bin')

run_name = "composed_tensor_test_1"
# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    # mlflow.log_params(params)
    #print(run.info)
    # Log the error metrics that were calculated during validation
    # mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.pytorch.log_state_dict(state_dict = sd, artifact_path = 'model')
    state_dict_uri = mlflow.get_artifact_uri(artifact_path)
    mlflow.log_artifact('../pytorch_model.txt')
print(state_dict_uri)

mlflow-artifacts:/295829569244001181/c1a37cd5ceb44c999fd70318edf5b41f/artifacts/model


In [6]:
state_dict = mlflow.pytorch.load_state_dict(state_dict_uri)

In [8]:
len(state_dict)

424

In [18]:
mlflow.tracking.get_tracking_uri(), mlflow.get_artifact_uri(),

('http://127.0.0.1:5000',
 'mlflow-artifacts:/295829569244001181/7fc1cd52475f46d68302167a1e0f45a7/artifacts')

In [ ]:
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Use the fluent API to set the tracking uri and the active experiment
mlflow.set_tracking_uri("http://127.0.0.1:8080")

# Sets the current active experiment to the "Apple_Models" experiment and returns the Experiment metadata
test_experiment = mlflow.set_experiment("Apple_Models")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "apples_rf_test"

# Define an artifact path that the model will be saved to.
artifact_path = "rf_apples"

In [30]:
# Split the data into features and target and drop irrelevant date field and target field
X = data.drop(columns=["date", "demand"])
y = data["demand"]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    "n_estimators": 100,
    "max_depth": 6,
    "min_samples_split": 10,
    "min_samples_leaf": 4,
    "bootstrap": True,
    "oob_score": False,
    "random_state": 888,
}

# Train the RandomForestRegressor
rf = RandomForestRegressor(**params)

# Fit the model on the training data
rf.fit(X_train, y_train)

# Predict on the validation set
y_pred = rf.predict(X_val)

# Calculate error metrics
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

# Assemble the metrics we're going to write into a collection
metrics = {"mae": mae, "mse": mse, "rmse": rmse, "r2": r2}

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    mlflow.log_params(params)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(sk_model=rf, input_example=X_val, artifact_path=artifact_path)

2023/11/16 10:13:07 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

/home/jy_zhang/anaconda3/lib/python3.11/site-packages/mlflow/models/signature.py:335: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing val

#### Success!

You've just logged your first MLflow model! 

Navigate to the MLflow UI to see the run that was just created (named "apples_rf_test", logged to the Experiment "Apple_Models"). 

In [31]:
import mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

mlflow.autolog()

db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)
rf.fit(X_train, y_train)

# Use the model to make predictions on the test dataset.
predictions = rf.predict(X_test)


2023/11/16 10:26:02 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/11/16 10:26:02 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'cdfae31843f04bab88271c1a08568d85', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
